In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../datasets_models/data.csv")

In [ ]:
# x < 6 Nada de lluvia
# 6 < x <= 12 Lluvia debil
# 12 < x <= 90 Lluvia moderada
# 90 < x <= 180 Lluvia fuerte
# 180 < x <= 360 Lluvia muy fuerte
# > 360 Lluvia torrencial

In [4]:
def label_encoder(x):
    if x < 6:
        return 0
    elif 6 < x <= 12:
        return 1
    elif 12 < x <= 90:
        return 2
    elif 90 < x <= 180:
        return 3
    elif 180 < x <= 360:
        return 4
    else:
        return 5

df['precip_encoded'] = df['precip'].apply(label_encoder)

0